In [1]:
!pip install opencv-python
!pip install imutils
!pip install scipy
!pip install playsound==1.2.2


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   --- ------------------------------------ 3.1/39.5 MB 15.4 MB/s eta 0:00:03
   ------ --------------------------------- 6.8/39.5 MB 16.1 MB/s eta 0:00:03
   ---------- ----------------------------- 10.2/39.5 MB 15.9 MB/s eta 0:00:02
   ------------- -------------------------- 13.6/39.5 MB 16.1 MB/s eta 0:00:02
   ----------------- ---------------------- 17.0/39.5 MB 16.3 MB/s eta 0:00:02
   -------------------- ------------------- 20.7/39.5 MB 16.2 MB/s eta 0:00:02
   ----------------------- ---------------- 23.1/39.5 MB 15.9 MB/s eta 0:00:02
   ------------------------- -------------- 25.2/39.5 MB 15.0 MB/s eta 0:00:01
   --------------------------- ------------ 27.0/39.5 MB 14.0 MB/s eta 0:00:01
   ----------------------------- ---------- 29.1/39.5 MB 13.7 MB/s eta 0:00:01
   ------------------------------- -------- 31.2/39.5 MB 13.4 MB/s eta 0:00:01
   --------------------------------- ------ 32.8/39.5 MB 12.8 M

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [41 lines of output]
  running bdist_wheel
  running build
  running build_ext
  
  
                     CMake is not installed on your system!
  
      Or it is possible some broken copy of cmake is installed on your system.
      It is unfortunately very common for python package managers to include
      broken copies of cmake.  So if the error above this refers to some file
      path to a cmake file inside a python or anaconda or miniconda path then you
      should delete that broken copy of cmake from your computer.
  
      Instead, please get an official copy of cmake from one of these known good
      sources of an official cmake:
          - cmake.org (this is how windows users should get cmake)
          - apt install cmake (for Ubuntu or Debian based systems)
          - yum install cmake (for Redhat or CenOS based systems)
  
      On a linux machine you c

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25857 sha256=4418a87214ae4e716fb58c9a7c5cdae6ebe2e025aa1c2c5ee400c2ba1024b819
  Stored in directory: c:\users\deepika\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils


In [10]:
%pip install mediapipe opencv-python numpy playsound==1.2.2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import time
from playsound import playsound



In [11]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils


In [12]:
# Eye and iris indices (MediaPipe)
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

In [13]:
# Head pose estimation
model_points = np.array([
    (0.0, 0.0, 0.0),           # Nose tip
    (0.0, -330.0, -65.0),      # Chin
    (-225.0, 170.0, -135.0),   # Left eye left corner
    (225.0, 170.0, -135.0),    # Right eye right corner
    (-150.0, -150.0, -125.0),  # Left mouth corner
    (150.0, -150.0, -125.0)    # Right mouth corner
], dtype=np.float64)
landmark_ids = [1, 152, 263, 33, 287, 57]

In [14]:
# EAR calculation
def euclidean_distance(pt1, pt2):
    return np.linalg.norm(np.array(pt1) - np.array(pt2))

def get_EAR(landmarks, eye_indices, w, h):
    points = [(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) for idx in eye_indices]
    A = euclidean_distance(points[1], points[5])
    B = euclidean_distance(points[2], points[4])
    C = euclidean_distance(points[0], points[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [15]:
# Iris position for gaze direction
def get_iris_position(iris_points, eye_points):
    cx_iris = np.mean([p[0] for p in iris_points])
    left_x = eye_points[0][0]
    right_x = eye_points[3][0]
    if cx_iris < left_x + (right_x - left_x) / 3:
        return "RIGHT"
    elif cx_iris > left_x + 2 * (right_x - left_x) / 3:
        return "LEFT"
    else:
        return "CENTER"

# Thresholds and counter
EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 30
counter = 0

In [16]:
# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    h, w = frame.shape[:2]
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # EAR
            left_ear = get_EAR(face_landmarks.landmark, LEFT_EYE, w, h)
            right_ear = get_EAR(face_landmarks.landmark, RIGHT_EYE, w, h)
            avg_ear = (left_ear + right_ear) / 2.0

            # Iris and eye coordinates
            landmarks = face_landmarks.landmark
            left_eye = [(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) for idx in LEFT_EYE]
            right_eye = [(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) for idx in RIGHT_EYE]
            left_iris = [(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) for idx in LEFT_IRIS]
            right_iris = [(int(landmarks[idx].x * w), int(landmarks[idx].y * h)) for idx in RIGHT_IRIS]

            # Gaze direction
            gaze_left = get_iris_position(left_iris, left_eye)
            cv2.putText(frame, f"Gaze: {gaze_left}", (30, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            # Drowsiness detection
            if avg_ear < EAR_THRESHOLD:
                counter += 1
                if counter > EAR_CONSEC_FRAMES:
                    cv2.putText(frame, "DROWSY ALERT!", (50, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
            else:
                counter = 0

            cv2.putText(frame, f"EAR: {avg_ear:.2f}", (30, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

            # Head pose (yaw) - distraction detection
            image_points = []
            for idx in landmark_ids:
                lm = face_landmarks.landmark[idx]
                x, y = int(lm.x * w), int(lm.y * h)
                image_points.append((x, y))

            image_points = np.array(image_points, dtype="double")
            focal_length = w
            center = (w / 2, h / 2)
            camera_matrix = np.array(
                [[focal_length, 0, center[0]],
                 [0, focal_length, center[1]],
                 [0, 0, 1]], dtype="double"
            )
            dist_coeffs = np.zeros((4, 1))

            success, rotation_vector, translation_vector = cv2.solvePnP(
                model_points, image_points, camera_matrix, dist_coeffs)

            rmat, _ = cv2.Rodrigues(rotation_vector)
            proj_mat = np.hstack((rmat, translation_vector))
            _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(proj_mat)
            yaw = euler_angles[1]

            if abs(yaw) > 30:
                cv2.putText(frame, "DISTRACTED!", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 165, 255), 3)

    cv2.imshow("Driver Monitoring System", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()